In [ ]:
.libPaths()

In [ ]:
require(MathBioSim)
library(foreach)
library(doParallel)

linespec = function(from, to, count) {
  return((0:(count - 1)) / count * (to - from) + from)
}

In [ ]:
file_name = "NEW_TEST"

In [3]:
#gen_title = function(par, sm, sw) {
#  return(sprintf("b = [%g, %g], d = [%g, %g], d` = [%g, %g, %g, %g], sm = [%g, %g], sw = [%g, %g, %g, %g]",
#                 par['b_1'], par['b_2'],
#                 par['d_1'], par['d_2'],
#                 par['dd_1_1'], par['dd_1_2'], par['dd_2_1'], par['dd_2_2'],
#                 sm[1], sm[2],
#                 sw[1, 1], sw[1, 2], sw[2, 1], sw[2, 2]
#                 ));
#}
#
#dem_file<-file(paste(file_name, ".dem", sep = ""));
#dem_data = sprintf("reset\nset key top left\nset term jpeg size 1920, 1080\nset output '%s.jpg'\n%sset grid\nset title '%s'\nplot '%s' u 1:2 w l title 'N1 simulation' lc 'red', %g w l lt 0 lc 'red' title 'N1 numeric', '%s' u 1:3 w l title 'N2 simulation' lc 'blue',  %g w l lt 0 lc 'blue' title 'N2 numeric', '%s' u 1:($2+$3) w l title 'sum simulation', %g w l lt 0 title 'sum numeric", file_name, ifelse(FALSE, "set yrange[-20: 250]\n", ''), gen_title(params, SM, SW), dat_file_path, N1_num, dat_file_path, N2_num, dat_file_path, N1_num + N2_num)
#writeLines(dem_data, dem_file);
#close(dem_file);

In [4]:
area_length = 200
cell_count_x = 100
x_grid_points = 5000

SPECIES_COUNT = 2;

In [6]:


b = c(0.4, 0.4)
d = c(0.2, 0.2)

dd = matrix(
    c(
        0.001, 0.001,
        0.001, 0.001
    ),
    nrow = SPECIES_COUNT,
    ncol = SPECIES_COUNT
)

sigma_m = c(0.04, 0.1)
sigma_w = matrix(
    c(
        0.04, 0.04,
        0.04, 0.04
    ),
    nrow = SPECIES_COUNT,
    ncol = SPECIES_COUNT
)

params<-list(
    "species_count"=SPECIES_COUNT,
    "area_length_x"=area_length,
    "cell_count_x"=cell_count_x,
    "seed"=42,
    "spline_precision"=1e-6,
    "periodic"=FALSE
)
  


In [7]:
for (i in 1:SPECIES_COUNT) {
    params[paste("b", i, sep = "_")] = b[i]
    params[paste("d", i, sep = "_")] = d[i]
    
    for (j in 1:(area_length * 100)) {
        params[paste("initial_population", "x", sep = "_")] = runif(1, 0, area_length)
        params["initial_population_species"] = i 
    }
    
    sigma = sigma_m[i]
    length = sigma * 5
    grid = linespec(0, length, x_grid_points)
    
    params[[paste("birth_kernel", "r", i, sep = "_")]] = length
    params[[paste("birth_kernel", "y", i, sep = "_")]] = dnorm(grid, sd = sigma)
    
    for (j in 1:SPECIES_COUNT) {
        params[paste("dd", i, j, sep = "_")] = dd[i, j]
        
        sigma = sigma_w[i, j]
        length = sigma * 5
        grid = linespec(0, length, x_grid_points)
        
        params[[paste("death_kernel", "r", i, j, sep = "_")]] = length
        params[[paste("death_kernel", "y", i, j, sep = "_")]] = dnorm(grid, sd = sigma)
    }
}

In [ ]:
d12 = 0.0010175

params["dd_1_2"] = d12
params["dd_2_1"] = d12

print(paste("Start: ", file_name))
dat_file_path = paste(file_name, "dat", "csv", sep = ".")
dis_file_path = paste(file_name, "dis", "csv", sep = ".")

total_time = 0
file.create(dis_file_path)
dat_file<-file(dat_file_path);
writeLines("event;N1;N2", dat_file);
close(dat_file);

time_start<-Sys.time();

sim = new(poisson_1d_n_species, params);

write_data = function() {
    pop = sim$total_population / area_length;
    for (j in 1:SPECIES_COUNT) {
        cat(sim$GetAllCoordsForSpecies(j - 1), "\n", file = dis_file_path, append = TRUE)
    }
    cat(sim$events, pop[1], pop[2], sep = "\t", file = dat_file_path, append = TRUE, fill = TRUE);
}

coef = 6 * 2 ** 10


time_start<-Sys.time();
for (k in 1:(5 * (2 ** 10))){
    sim$run_events(coef)
    write_data()
}
time_finish<-Sys.time();
total_time = total_time + time_finish - time_start;
print(total_time);



[1] "Start:  NEW_TEST"


[1] TRUE

In [24]:
time_start<-Sys.time();
for (k in 1:(5 * (2 ** 10))){
    sim$run_events(coef)
    write_data()
}
time_finish<-Sys.time();
total_time = total_time + time_finish - time_start;
print(total_time);



Time difference of 2.889468 hours


ERROR: Error in .getClassesFromCache(Class): объект 'poisson_1d_n_species' не найден
